<h1> Discounted Cash Flow Model</h1>
<h2> Google (Alphabet Inc.) Financial Data</h2>
<h3> A project to develop a robust and reuseable script for calculating DCF and Sensitivity Analysis</h3>

<p>
Our project will follow these key steps:

1. Gather Historical Data: We'll fetch several years of financial statements to understand historical performance.
2. Forecast Financial Statements: We'll project key line items like revenue and expenses for the next 5 years.
3. Calculate Free Cash Flow (FCF): We'll calculate the cash available to investors for each of our forecast years, using the Gordon formula.
4. Calculate Terminal Value: We'll estimate the value of all cash flows beyond our 5-year forecast period.
5. Discount to Present Value: We'll use a discount rate (WACC) to bring all future cash flows back to today's value to get our final valuation.
6. Sensitivity Analysis: We'll vary the WACC and PGR by a step up to shows how the final valuation changes based on a range these different assumptions.

</p>

<h3> 1. Setup and Data Fetching </h3>

In [23]:
import yfinance as yf
import pandas as pd
import numpy as np

# -==> 1. SETUP AND DATA GATHERING <==-

# I'm going to start by picking Google as my example company and set the projection to 5 years
TICKER = 'GOOGL'  # Change this to any ticker you want to analyze
PROJECTION_YEARS = 5

print(f"-==> Starting DCF Analysis for {TICKER} <==-")
print(f"Fetching historical data for the last {PROJECTION_YEARS} years...")

try:
    ticker = yf.Ticker(TICKER)
    
    # I'll get 5 years of historical financial statements.
    income_statement_df = ticker.financials.iloc[:, :PROJECTION_YEARS]
    balance_sheet_df = ticker.balance_sheet.iloc[:, :PROJECTION_YEARS]
    
    if income_statement_df.empty or balance_sheet_df.empty:
        raise ValueError("Could not retrieve financial data from yfinance.")
    
    # I need to transpose the data so that my rows are the years and my columns are the line items, makes analysis easier.
    income_statement_df = income_statement_df.T
    balance_sheet_df = balance_sheet_df.T

    print("Data fetched successfully!")

    # I need to know the column names, so i know how to specifically select the ones i need later.
    #print("\n--- Income Statement Column Names ---")
    #print(income_statement_df.columns.to_list())

    #print("\n--- Balance Sheet Column Names ---")
    #print(balance_sheet_df.columns.to_list())
    
except Exception as e:
    print(f"An error occurred while fetching data: {e}")
    # I'll exit the script if I can't get the data.
    exit()

-==> Starting DCF Analysis for GOOGL <==-
Fetching historical data for the last 5 years...
Data fetched successfully!


<h3> 2. Forecast Financials and free cash flow</h3>

<p>
Now that I have my historical data, I'll make a series of assumptions to forecast my financial statements. The most critical part here is calculating the Free Cash Flow (FCF), which is the cash a company generates after accounting for all capital expenditures and working capital changes.

I'll use a simple approach:

1. Revenue Growth: I'll assume a constant growth rate based on historical data.

2. COGS & Operating Expenses: I'll project these as a percentage of revenue, using historical averages.

3. Capital Expenditures (CapEx): I'll assume this is a percentage of revenue.

4. Depreciation & Amortization (D&A): I'll assume D&A is a percentage of Property, Plant & Equipment (PP&E).

In [24]:
# Module 2: Forecasting Financials and FCF
# -==> 2. FORECASTING FINANCIALS <==-

# In a case where we cant go through the financials to geth their column names, we can use this function to perform a fuzzy search for the key names.
def get_historical_value(df, key_list):
    """
    Safely extracts a historical value from a transposed DataFrame.
    It checks for partial, case-insensitive matches for a list of keys in the columns.
    If a value is not found or is NaN, it returns 0.
    """
    df_columns = [str(i).lower() for i in df.columns]
    for key in key_list:
        for i, col_name in enumerate(df_columns):
            if key.lower() in col_name:
                value = df.loc[df.index[0], df.columns[i]]
                if pd.isna(value):
                    return 0
                return value
    return 0


# In my case, i know the exact names of the cols. so i would select them directly.
# But in a case where you dont know the exact names, you can use the get_historical_value function to perform a fuzzy search.
# For example, if you want to get the total revenue, you can do:
# base_revenue = get_historical_value(income_statement_df, ['Total Revenue'])
base_revenue = get_historical_value(income_statement_df, ['Total Revenue'])
base_cogs = get_historical_value(income_statement_df, ['Cost Of Revenue'])
base_opex = get_historical_value(income_statement_df, ['Selling General And Administration'])
base_da = get_historical_value(income_statement_df, ['Reconciled Depreciation'])
base_net_income = get_historical_value(income_statement_df, ['Net Income'])
base_ppne = get_historical_value(balance_sheet_df, ['Net PPE'])

# These are my assumptions. I can change these to see how they impact the valuation( insensitivity analysis).
# I'll calculate historical means where possible, adding checks to prevent division by zero.
revenue_growth_rate = 0.10 # I'll assume a 10% growth rate for the projection period.
cogs_as_pct_revenue = base_cogs / base_revenue if base_revenue != 0 else 0.5
opex_as_pct_revenue = base_opex / base_revenue if base_revenue != 0 else 0.2
capex_as_pct_revenue = 0.05 # I'll assume a 5% CapEx on revenue.
tax_rate = 0.25 # I'll use a standard corporate tax rate of 25%.
depreciation_as_pct_ppne = base_da / base_ppne if base_ppne != 0 else 0.10 # I'll assume D&A is a percentage of PP&E.

# I'll also need a working capital assumption.
nwc_change_as_pct_revenue = 0.02 # I'll assume a 2% change in Net Working Capital.

# Now, I'll build my forecast.
projected_revenue = [base_revenue * (1 + revenue_growth_rate)**i for i in range(1, PROJECTION_YEARS + 1)]

# I'll create a list to store my free cash flows.
free_cash_flows = []

print("\n-==> Calculating Projected Financials and FCF <==-")
for year in range(PROJECTION_YEARS):
    # I'll calculate EBIT (Earnings Before Interest and Taxes).
    cogs = projected_revenue[year] * cogs_as_pct_revenue
    opex = projected_revenue[year] * opex_as_pct_revenue
    ebit = projected_revenue[year] - cogs - opex
    
    # Then I'll calculate NOPAT (Net Operating Profit After Tax).
    nopat = ebit * (1 - tax_rate)
    
    # I'll calculate D&A and CapEx.
    # Note: A more complex model would project PP&E and then D&A, but this is a good start.
    depreciation = base_ppne * depreciation_as_pct_ppne
    capex = projected_revenue[year] * capex_as_pct_revenue
    
    # I'll calculate the change in Net Working Capital (NWC).
    nwc_change = projected_revenue[year] * nwc_change_as_pct_revenue
    
    # Finally, I can calculate the FCF for this year.
    fcf = nopat + depreciation - capex - nwc_change
    free_cash_flows.append(fcf)
    
    print(f"Year {year+1} FCF: ${fcf:,.2f}")


#=================================================================
#Show a structured table of this
#=================================================================
fcf_df = pd.DataFrame(
    {'Year': [f'Year {i+1}' for i in range(len(free_cash_flows))], 'Free Cash Flow': free_cash_flows}
)

# Apply professional styling
styled_fcf = fcf_df.style.set_caption("Projected Free Cash Flows")\
    .format({'Free Cash Flow': "${:,.2f}"})\
    .hide(axis='index')\
    .set_properties(**{'background-color': '#f8f9fa', 'color': 'black', 'border-color': '#dee2e6'})\
    .set_table_styles([{'selector': 'th', 'props': [('background-color', '#343a40'), ('color', 'white'), ('border-color', '#dee2e6')]}])

display(styled_fcf)




-==> Calculating Projected Financials and FCF <==-
Year 1 FCF: $121,775,314,000.00
Year 2 FCF: $132,421,745,400.00
Year 3 FCF: $144,132,819,940.00
Year 4 FCF: $157,015,001,934.00
Year 5 FCF: $171,185,402,127.40


Year,Free Cash Flow
Year 1,"$121,775,314,000.00"
Year 2,"$132,421,745,400.00"
Year 3,"$144,132,819,940.00"
Year 4,"$157,015,001,934.00"
Year 5,"$171,185,402,127.40"


<h2>3. Calculating terminal Value and WACC </h2>
This is where things get a bit more theoretical. After my 5-year forecast, 
I need to assume that the company will continue to generate cash flows indefinitely.
 The value of these cash flows from year 6 onwards is called the Terminal Value.

I'll use the Gordon Growth Model for this calculation.
 The formula is: Terminal Value = FCF(n+1) / (WACC - g).

- FCF(n+1) is the first year's cash flow after my forecast period.
- WACC is the Weighted Average Cost of Capital, which is my discount rate.
- g is the perpetual growth rate, which is a small, constant growth rate (e.g., 2%).

In [25]:
# dcf_analysis.py - Module 3: Calculating Terminal Value and WACC

# --- 3. CALCULATING TERMINAL VALUE AND WACC ---

# I need to define my discount rate and perpetual growth rate.
# WACC is often calculated separately, but for this project, I'll use an assumed rate.
WACC = 0.08 # I'll assume an 8% WACC.
perpetual_growth_rate = 0.02 # I'll assume a 2% perpetual growth rate.

print("\n--- Calculating Terminal Value ---")

# First, I need to get the FCF for year n+1 (in our case, year 6).
fcf_year_n_plus_1 = free_cash_flows[-1] * (1 + perpetual_growth_rate)

# Now, I'll use the Gordon Growth Model to calculate the Terminal Value.
terminal_value = fcf_year_n_plus_1 / (WACC - perpetual_growth_rate)
print(f"Terminal Value: ${terminal_value:,.2f}")


--- Calculating Terminal Value ---
Terminal Value: $2,910,151,836,165.80


<h2>4. Discounting and final valuation</h2>
In this final step,
 I'll discount all my future cash flows
 the PROJECTION_YEARS of FCFs 
 and the Terminal Value—back to today's value using the WACC.

The formula for present value is: PV = FV / (1 + r)^n.

FV is the Future Value (the cash flow in a given year).

r is the discount rate (WACC).

n is the number of years into the future.

After summing all the present values, 
I'll have my company's total intrinsic value, 
which I can compare to its current market capitalization to see if it's over or undervalued.

In [26]:
# dcf_analysis.py - Module 4: Discounting and Final Valuation

# --- 4. DISCOUNTING AND FINAL VALUATION ---

# I'll create a list to store my discounted cash flows.
discounted_cash_flows = []

print("\n--- Discounting Cash Flows to Present Value ---")

# I'll discount each FCF from my forecast period.
for year, fcf in enumerate(free_cash_flows):
    discounted_fcf = fcf / (1 + WACC)**(year + 1)
    discounted_cash_flows.append(discounted_fcf)
    print(f"Year {year+1} Discounted FCF: ${discounted_fcf:,.2f}")

# I need to discount the terminal value as well.
# It gets discounted from the end of the last projection year.
discounted_terminal_value = terminal_value / (1 + WACC)**PROJECTION_YEARS
print(f"Discounted Terminal Value: ${discounted_terminal_value:,.2f}")

# Now I'll sum everything up to get the total Enterprise Value.
enterprise_value = sum(discounted_cash_flows) + discounted_terminal_value
print("\n--- Final Valuation ---")
print(f"Total Enterprise Value: ${enterprise_value:,.2f}")

# To get the Equity Value, I need to add cash and subtract debt.
# I will fetch these values from the balance sheet.
# The `get_historical_value` function from Module 2 is used here.
cash_and_equivalents = get_historical_value(
    balance_sheet_df,
      ['Cash And Cash Equivalents', 'Cash Cash Equivalents And Short Term Investments'])
total_debt = get_historical_value(balance_sheet_df, ['Total Debt'])

equity_value = enterprise_value - total_debt + cash_and_equivalents
print(f"Equity Value: ${equity_value:,.2f}")



# Finally, let's get the share price.
shares_outstanding = ticker.info.get('sharesOutstanding', 0)
if shares_outstanding:
    price_per_share = equity_value / shares_outstanding
    print(f"Implied Price Per Share: ${price_per_share:,.2f}")
    
    current_price = ticker.info.get('currentPrice', 0)
    print(f"Current Market Price Per Share: ${current_price:,.2f}")

    if price_per_share > current_price:
        print("\nConclusion: The model suggests the company may be undervalued. 🚀")
    elif price_per_share < current_price:
        print("\nConclusion: The model suggests the company may be overvalued. 📉")
    else:
        print("\nConclusion: The model suggests the company is fairly valued. ✅")
else:
    print("\nCould not find shares outstanding. Cannot calculate price per share.")


#====================================================
#show the table
#====================================================
# --- Final Valuation Summary Table ---
valuation_summary = pd.DataFrame({
    'Metric': [
        'Total Enterprise Value',
        'Equity Value',
        'Implied Price Per Share',
        'Current Market Price'
    ],
    'Value': [
        enterprise_value,
        equity_value,
        price_per_share,
        current_price
    ]
})
styled_summary = valuation_summary.style.set_caption("Final DCF Valuation Summary")\
    .format({'Value': "${:,.2f}"})\
    .hide(axis='index')\
    .set_properties(**{'background-color': '#f8f9fa', 'color': 'black', 'border-color': '#dee2e6'})\
    .set_table_styles([{'selector': 'th', 'props': [('background-color', '#343a40'), ('color', 'white'),
                                                     ('border-color', '#dee2e6')]}])

display(styled_summary)

#====================================================


--- Discounting Cash Flows to Present Value ---
Year 1 Discounted FCF: $112,754,920,370.37
Year 2 Discounted FCF: $113,530,302,983.54
Year 3 Discounted FCF: $114,417,279,457.53
Year 4 Discounted FCF: $115,410,713,758.38
Year 5 Discounted FCF: $116,505,908,265.37
Discounted Terminal Value: $1,980,600,440,511.37

--- Final Valuation ---
Total Enterprise Value: $2,553,219,565,346.56
Equity Value: $2,551,224,565,346.56
Implied Price Per Share: $438.58
Current Market Price Per Share: $201.00

Conclusion: The model suggests the company may be undervalued. 🚀


Metric,Value
Total Enterprise Value,"$2,553,219,565,346.56"
Equity Value,"$2,551,224,565,346.56"
Implied Price Per Share,$438.58
Current Market Price,$201.00


<h2>Conclusion: Final DCF Valuation</h2>
Based on the Discounted Cash Flow (DCF) model, 
the company's valuation suggests it may be undervalued.
The model's implied share price is significantly higher than the current market price, 
indicating a potential investment opportunity.

Implied Price Per Share: $371.82

Current Market Price Per Share: $201.00

This analysis suggests that the company's future cash flow generation,
when discounted back to the present, 
warrants a higher valuation than what the market is currently assigning it.

<h1>Sensitivity Analysis </h1>
A sensitivity analysis is a hallmark of a professional financial model. 
It shows how the final valuation changes based on a range of different assumptions (e.g., varying the WACC and the perpetuity growth rate). 
This demonstrates that you understand the assumptions driving your model and are transparent about its potential outcomes.


In [27]:
# This is a simplified example of how you would set up a sensitivity table.
# You would need to run your DCF calculation for each combination of WACC and growth rate.

wacc_range = np.arange(0.07, 0.11, 0.01) # 7% to 10%
growth_range = np.arange(0.01, 0.03, 0.005) # 1% to 2.5%

sensitivity_table = pd.DataFrame(index=[f"{g:.1%}" for g in growth_range], columns=[f"{w:.1%}" for w in wacc_range])

for g in growth_range:
    for w in wacc_range:
        # Re-run your final valuation calculation with the new g and w
        # (This logic is for demonstration; your actual code would be more complex)
        fcf_year_n_plus_1 = free_cash_flows[-1] * (1 + g)
        if (w - g) > 0:
            terminal_value = fcf_year_n_plus_1 / (w - g)
            discounted_tv = terminal_value / (1 + w)**PROJECTION_YEARS
            total_pv = sum([fcf / (1 + w)**(i + 1) for i, fcf in enumerate(free_cash_flows)]) + discounted_tv
            sensitivity_table.loc[f"{g:.1%}", f"{w:.1%}"] = total_pv
        else:
            sensitivity_table.loc[f"{g:.1%}", f"{w:.1%}"] = np.nan

# Display the styled sensitivity table
styled_sensitivity = sensitivity_table.style.background_gradient(cmap='YlGnBu')\
    .format("${:,.2f}")\
    .set_caption("Sensitivity Analysis: Enterprise Value by WACC & Growth Rate")

display(styled_sensitivity)

,7.0%,8.0%,9.0%,10.0%
1.0%,"$2,643,521,021,263.58","$2,253,632,944,092.74","$1,961,608,256,631.33","$1,734,809,358,905.66"
1.5%,"$2,841,394,541,953.62","$2,391,903,692,363.73","$2,062,668,302,159.15","$1,811,228,922,967.81"
2.0%,"$3,078,842,766,781.66","$2,553,219,565,346.56","$2,178,165,497,048.09","$1,897,200,932,537.74"
2.5%,"$3,369,057,263,793.71","$2,743,865,597,053.54","$2,311,431,491,150.71","$1,994,635,876,716.98"
